# Check for GPU in the colab using tensorflow

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# To get the allocated GPU spec data in google Colab

In [ ]:
!nvidia-smi

Wed Jun  7 07:37:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |    387MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Building a Neural Network with PyTorch

In [ ]:

#imported the dependencies

import torch                                  
import torchvision

from torch import nn , save,load                #neural network class: neural network methods 
from torch.optim import Adam                    #Optimiser class
from torch.utils.data import DataLoader         #Load the data from pytorch

from torchvision import datasets                #datasets
from torchvision.transforms import ToTensor     #convert images into tensors to work with pytorch

#download the dataset: "MNIST" Image classification dataset 10 classes 0-9 using datasets

train = datasets.MNIST(root="data", download=True,train=True, transform=ToTensor())   
                      #'root="data"' where to download folder name 'data'
                      #'download = True' to download the data to the folder
                      #'train=True' to train partition
                      #'transform=ToTensor' to specify any of the transition to be applied

dataset = DataLoader(train, 32)                #pass the data downloaded to the loader and specify how to convert it to batches of images
                      #batches of 32 images : image shape : 1,28,28 pixels

#create the neural network Image classifier class

class ImageClassifier(nn.Module):              #class created sub classified from neural network.module class
  #two functions
  def __init__(self):                              #definite function
    #subclass this model
    super().__init__()
    #create a model where the layers are stack overed
    self.model = nn.Sequential(                #creating new attribute using Sequential API from pytorch.nn
        #1st convolutional neural network layer
        nn.Conv2d(1 , 32,(3,3)),               #convolutional neural network(input kernal = 1(black and white images), filter = 32, filter_size = (3X3)) 3 by 3 of 32 filters
        nn.ReLU(),                             #Activations to handle with the non-linearities
        #2nd Convolutional layer
        nn.Conv2d(32,64,(3,3)),                #input=32 , output=64
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        #by applying each on of these layers with these specific parameters will shave off 2 pixels of height and width of each image
        #to adjust that that in the output layer we apply a linear layer

        nn.Flatten(),                          #1st flatten the layers
        nn.Linear(64*(28-6)*(28-6), 10)        #then create applying linear layers passing the input shape
                 #(last convolutional final output*(pixel_size-shave_off_size(2pixceleach*3convolutions = 6))*(pixel_size-shave_off_size(2pixceleach*3convolutions = 6)),output shape need to be the no. of classes)
    )

  def forward(self,x):                         #forward function call method inside tensor flow
    #take in current instance and x data
    return self.model(x)                       #return the value

#Create an instance of the neural network,loss and optimizer

clf = ImageClassifier().to('cuda')             #instance of neural network
opt = Adam(clf.parameters(), lr = 1e-3)        #instanciating the optimizer with the parameters and learning how fast or slow
loss_fn= nn.CrossEntropyLoss()                 #loss function

#Training flow
if __name__ == "__main__":
  for epoch in range(10):                      #training for 10 epochs
    for batch in dataset:                      #looping through the dataset loaded
      X,y = batch                              #unbatch the data
      X,y = X.to('cuda'),y.to('cuda')          #send X and y value to GPU again
      yhat = clf(X)                            #make predictions
      loss = loss_fn(yhat,y)                   #calculate loss

      #In Tensor flow go compile and got fit
      #apply backprop

      opt.zero_grad()                          #zero grade any of the exsisting gradients
      loss.backward()                          #calculate the gradients
      opt.step()                               #Apply gradient descent
    print(f"Epoch: {epoch} Loss is {loss}")    #printing the loss and epoch

with open('model_state.pt','wb') as f:
  save(clf.state_dict(),f)


Epoch: 0 Loss is 0.04301353916525841
Epoch: 1 Loss is 0.008015292696654797
Epoch: 2 Loss is 0.0006723475526086986
Epoch: 3 Loss is 5.037411392549984e-05
Epoch: 4 Loss is 8.47060073283501e-06
Epoch: 5 Loss is 2.8300632038735785e-05
Epoch: 6 Loss is 7.934841050882824e-07
Epoch: 7 Loss is 1.5385203369078226e-06
Epoch: 8 Loss is 1.1287438610452227e-06
Epoch: 9 Loss is 0.00011815538891823962


In [6]:
import torch 
import PIL
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Get data 
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)
#1,28,28 - classes 0-9

# Image Classifier Neural Network
class ImageClassifier(nn.Module): 
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)), 
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)), 
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)), 
            nn.ReLU(),
            nn.Flatten(), 
            nn.Linear(64*(28-6)*(28-6), 10)  
        )

    def forward(self, x): 
        return self.model(x)

# Instance of the neural network, loss, optimizer 
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss() 

# Training flow 
if __name__ == "__main__": 
   with open('model_state.pt','rb') as f:
    clf.load_state_dict(load(f))

   img = Image.open('img_3.jpg')
   img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

   print(torch.argmax(clf(img_tensor)))

tensor(9, device='cuda:0')
